In [ ]:
#pip install geopy 

In [ ]:
#pip install Nominatim

In [15]:
#from geopy.geocoders import Nominatim
import requests
from config import API_key
import pandas as pd
from pprint import pprint
from json.decoder import JSONDecodeError

In [2]:
summary = "Data/summary.csv"
summary = pd.read_csv(summary)
summary.head()

Unnamed: 0           Date            City State Country    Shape  \
0           1  7/31/22 21:59  West Lafayette    OH     USA    Light   
1           2  7/31/22 21:45      Burr Ridge    IL     USA    Other   
2           4  7/31/22 21:15          Conway    SC     USA   Circle   
3           5  7/31/22 21:11          Keyser    WV     USA    Light   
4           6  7/31/22 20:43        gressham    OR     USA  Unknown   

             Duration                                            Summary  \
0           4 minutes            Bright circle light moving through sky.   
1                 NaN  Strange red, blue and black craft in the sky a...   
2  Less than a minute  Round bright white light go in a straight line...   
3           2 minutes                Saw an orb that interacted with me.   
4  less than a minute  A single soft red light that pulsed three time...   

   Posted Images  ReportNum  
0  9/9/22    Yes          1  
1  9/9/22    Yes          2  
2  9/9/22    NaN          4  
3  9/9/22    NaN          5  
4  9/9/22    NaN          6

In [3]:
summary.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
summary.drop(["a"], axis=1, inplace=True)

In [4]:
city_df = summary[['City', 'State', 'ReportNum']]

In [5]:
city_df.fillna("")

City State  ReportNum
0     West Lafayette    OH          1
1         Burr Ridge    IL          2
2             Conway    SC          4
3             Keyser    WV          5
4           gressham    OR          6
...              ...   ...        ...
2126       Snowflake    AZ        153
2127                              154
2128         El paso    TX        155
2129     CHARLESTOWN    IN        156
2130       Mansfield    TX        157

[2131 rows x 3 columns]

In [6]:
city_df["city/state"] = city_df[['City', 'State']].apply(lambda x: ', '.join(x[x.notnull()]), axis=1)

C:\Users\13135\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [7]:
city_df.head()

City State  ReportNum          city/state
0  West Lafayette    OH          1  West Lafayette, OH
1      Burr Ridge    IL          2      Burr Ridge, IL
2          Conway    SC          4          Conway, SC
3          Keyser    WV          5          Keyser, WV
4        gressham    OR          6        gressham, OR

In [8]:
uniques = city_df['city/state'].unique()
unique_df = pd.DataFrame(uniques)
unique_df

0
0     West Lafayette, OH
1         Burr Ridge, IL
2             Conway, SC
3             Keyser, WV
4           gressham, OR
...                  ...
1673       Blakeslee, OH
1674       Snowflake, AZ
1675         El paso, TX
1676     CHARLESTOWN, IN
1677       Mansfield, TX

[1678 rows x 1 columns]

In [9]:
unique_df[['City', 'State']] = unique_df[0].str.split(', ', 1, expand=True)

In [10]:
unique_df.head()

0            City State
0  West Lafayette, OH  West Lafayette    OH
1      Burr Ridge, IL      Burr Ridge    IL
2          Conway, SC          Conway    SC
3          Keyser, WV          Keyser    WV
4        gressham, OR        gressham    OR

In [11]:
unique_df.rename({0:"Location"}, axis="columns", inplace=True)

In [12]:
unique_df["Latitude"]=""
unique_df["Longitude"]=""
unique_df

Location            City State Latitude Longitude
0     West Lafayette, OH  West Lafayette    OH                   
1         Burr Ridge, IL      Burr Ridge    IL                   
2             Conway, SC          Conway    SC                   
3             Keyser, WV          Keyser    WV                   
4           gressham, OR        gressham    OR                   
...                  ...             ...   ...      ...       ...
1673       Blakeslee, OH       Blakeslee    OH                   
1674       Snowflake, AZ       Snowflake    AZ                   
1675         El paso, TX         El paso    TX                   
1676     CHARLESTOWN, IN     CHARLESTOWN    IN                   
1677       Mansfield, TX       Mansfield    TX                   

[1678 rows x 5 columns]

In [16]:
#API call
for index,row in unique_df.iterrows():
    city = row["Location"]
    url = "https://maps.googleapis.com/maps/api/geocode/json?address=" 
    print(f"Processing Record for {city}")
    try: 
        query_url = f"{url}{city}&key={API_key}"
        response = requests.get(query_url).json()
#         pprint(response)
        unique_df.loc[index,"Latitude"] = response['results'][0]['geometry']['location']['lat']
        unique_df.loc[index,"Longitude"]= response['results'][0]['geometry']['location']['lng']
#         city_name.append(response['results'][0][address_components][0][long_name])
#         city_name.append(city)
    except(KeyError, IndexError, JSONDecodeError):
        print(f"City not found. Skipping...")
       

Processing Record for West Lafayette, OH
Processing Record for Burr Ridge, IL
Processing Record for Conway, SC
Processing Record for Keyser, WV
Processing Record for gressham, OR
Processing Record for Eddyville, KY
Processing Record for Carpinteria, CA
Processing Record for Tacoma, WA
Processing Record for Charlotte, NC
Processing Record for Burnet, TX
Processing Record for Seattle, WA
Processing Record for Naples, FL
Processing Record for Flemington, NJ
Processing Record for Fremont, CA
Processing Record for Bella Vista, AR
Processing Record for Dorchester, MA
Processing Record for Dalton Gardens, ID
Processing Record for Philadelphia, PA
Processing Record for Harrisburg, PA
Processing Record for Wexford, PA
Processing Record for Newark, NJ
Processing Record for Hicksville, NY
Processing Record for Chenoa
Processing Record for River Forest, IL
Processing Record for Eudora, KS
Processing Record for North Dartmouth, MA
Processing Record for Randolph, VT
Processing Record for Maupin, OR


Processing Record for Hollywood, FL
Processing Record for Jupiter, FL
Processing Record for Marshall, TX
Processing Record for Phoenix, AZ
Processing Record for Norwalk, CT
Processing Record for Smith Mills, KY
Processing Record for YPSILANTI, MI
Processing Record for Fripp Island, SC
Processing Record for Moffat, CO
Processing Record for Morganton, NC
Processing Record for New Smyrna Beach, FL
Processing Record for Jefferson City, TN
Processing Record for Garland, TX
Processing Record for Ewing, NJ
Processing Record for Huntley, MT
Processing Record for Hazel Green, WI
Processing Record for Smyrna, TN
Processing Record for Dayton, NV
Processing Record for Beaver Island, MI
Processing Record for CROSS, SC
Processing Record for Louisville, KY
Processing Record for Lawrence, MA
Processing Record for Titonka, IA
Processing Record for West Deptford, NJ
Processing Record for San Augustine, TX
Processing Record for Greybull, WY
Processing Record for Chesterfield, MO
Processing Record for Seb

Processing Record for Tiverton, RI
Processing Record for Royal, IA
Processing Record for Culver City, CA
Processing Record for Crawfordsville, IN
Processing Record for Imperial, MO
Processing Record for West haven, CT
Processing Record for SCOTTSVILLE, VA
Processing Record for Fulton, MO
Processing Record for Elkton, MD
Processing Record for Wilmington, OH
Processing Record for Clearwater, FL
Processing Record for Atlantic City, NJ
Processing Record for Springhill, FL
Processing Record for Lisbon, ME
Processing Record for Beloit, WI
Processing Record for Scranton, PA
Processing Record for Circleville, OH
Processing Record for North Ferrisburgh, VT
Processing Record for Chillicothe, OH
Processing Record for South Phoenix, AZ
Processing Record for West Milford, NJ
Processing Record for New York, NY
Processing Record for Portola, CA
Processing Record for Spokane, WA
Processing Record for Kenmore, WA
Processing Record for Oregon City, OR
Processing Record for Doylestown, PA
Processing Reco

Processing Record for Weatherly, PA
Processing Record for Crystal City, Arlington, VA
Processing Record for Lexington, KY
Processing Record for West Springfield, MA
Processing Record for Over Toledo bend lake, TX
Processing Record for Lebanon, OH
Processing Record for Gridley, CA
Processing Record for Vineyard, UT
Processing Record for GROVE CITY, OH
Processing Record for Hickory, NC
Processing Record for Conifer, CO
Processing Record for Itasca, TX
Processing Record for Lebanon, TN
Processing Record for Friant, CA
Processing Record for Mt. Pleasant, WI
Processing Record for Godley, TX
Processing Record for Franklin, IN
Processing Record for Caldwell, ID
Processing Record for BLAIRSVILLE, PA
Processing Record for Erie, Presque Isle, PA
Processing Record for High Springs, FL
Processing Record for Garden City, SC
Processing Record for Lanesville, IN
Processing Record for Murrieta, CA
Processing Record for Michigan City, IN
Processing Record for Truchas, NM
Processing Record for Kensingto

Processing Record for Rural/Not Applicable, WV
City not found. Skipping...
Processing Record for Arlington, WA
Processing Record for Virgin, UT
Processing Record for Westland, MI
Processing Record for Greenville, NC
Processing Record for Shorewood, WI
Processing Record for Phoenixville, PA
Processing Record for West Bloomfield, NY
Processing Record for Ocean City, MD
Processing Record for between lindale texas and canton texas, TX
Processing Record for Saint Augustine, FL
Processing Record for Vista, CA
Processing Record for Arcola, TX
Processing Record for Valdosta, GA
Processing Record for Dallas, GA
Processing Record for Summerville, SC
Processing Record for McCormick, SC
Processing Record for Lackawanna, NY
Processing Record for Dothan, AL
Processing Record for Champaign, IL
Processing Record for Fowler, CA
Processing Record for Jamestown, TN
Processing Record for Winterport, ME
Processing Record for Fairbanks, AK
Processing Record for Palo Alto, CA
Processing Record for Douglasvil

Processing Record for New Port Richey, FL
Processing Record for Speculator, NY
Processing Record for Show Low, AZ
Processing Record for Saint joe, IN
Processing Record for McMinnville, OR
Processing Record for Panama city, FL
Processing Record for Arvada, CO
Processing Record for Davnport. Fl, FL
Processing Record for Homosassa, FL
Processing Record for Rrochester, NH
Processing Record for Center Point, IA
Processing Record for Winter Park
Processing Record for Lampasas County, TX
Processing Record for Mason City, IA
Processing Record for Annapolis, MD
Processing Record for Benson, NC
Processing Record for Tampa, FL
Processing Record for Jamestown, NY
Processing Record for Valley View, PA
Processing Record for Topeka, KS
Processing Record for Truxton, NY
Processing Record for North Royalton, OH
Processing Record for Altmar, NY
Processing Record for Melrose, MA
Processing Record for Port charlotte, FL
Processing Record for Yucaipa, CA
Processing Record for Oswego, IL
Processing Record f

Processing Record for Brooksville, FL
Processing Record for Toronto, ON
Processing Record for St Augustine, FL
Processing Record for Wyomissing, PA
Processing Record for milwaukee, WI
Processing Record for Paris, TX
Processing Record for westminster, CA
Processing Record for Newington, CT
Processing Record for Diablo/Martinez, CA
Processing Record for HILO, HI
Processing Record for New York
Processing Record for wauchula and myakka city, FL
Processing Record for Bear, DE
Processing Record for Berwick, PA
Processing Record for Des Moines, IA
Processing Record for Raynham, MA
Processing Record for Corte Madera, CA
Processing Record for Merrill, WI
Processing Record for Egg harbor, WI
Processing Record for Bealeton, VA
Processing Record for Addison, NY
Processing Record for Melbourne, FL
Processing Record for Jacksonville, NC
Processing Record for Wasco, CA
Processing Record for Vinton, VA
Processing Record for Brent, AL
Processing Record for New York q, NY
Processing Record for Centrevil

Processing Record for St George, KS
Processing Record for Rocksprings, TX
Processing Record for Murray, KY
Processing Record for Bogart
City not found. Skipping...
Processing Record for BROOKLYN CENTER, MN
Processing Record for Lafayette, PA
Processing Record for Lamcaster, SC
Processing Record for Westport, MA
Processing Record for Linton, IN
Processing Record for Stroudsburg, PA
Processing Record for Saltville, VA
Processing Record for Stanley, NY
Processing Record for Stoutsville, OH
Processing Record for Rio Rancho, NM
Processing Record for Murphy, NC
Processing Record for Monroeville, PA
Processing Record for Norman, OK
Processing Record for Lititz, PA
Processing Record for Germantown, NY
Processing Record for North east, MD
Processing Record for lebanon, CT
Processing Record for South Portland, ME
Processing Record for Douglas/Bill, WY
Processing Record for Wesley Chapel, FL
Processing Record for FRANKLIN, CT
Processing Record for Sunrise, FL
Processing Record for Inverness, FL
P

In [17]:
unique_df

Location            City State Latitude Longitude
0     West Lafayette, OH  West Lafayette    OH  40.2753   -81.751
1         Burr Ridge, IL      Burr Ridge    IL  41.7489  -87.9184
2             Conway, SC          Conway    SC   33.836  -79.0478
3             Keyser, WV          Keyser    WV  39.4409  -78.9739
4           gressham, OR        gressham    OR  45.5045  -122.436
...                  ...             ...   ...      ...       ...
1673       Blakeslee, OH       Blakeslee    OH  41.5242  -84.7302
1674       Snowflake, AZ       Snowflake    AZ  34.5134  -110.078
1675         El paso, TX         El paso    TX  31.7619  -106.485
1676     CHARLESTOWN, IN     CHARLESTOWN    IN  38.4531  -85.6702
1677       Mansfield, TX       Mansfield    TX  32.5632  -97.1417

[1678 rows x 5 columns]

In [19]:
summary

Date            City State Country     Shape  \
0     7/31/22 21:59  West Lafayette    OH     USA     Light   
1     7/31/22 21:45      Burr Ridge    IL     USA     Other   
2     7/31/22 21:15          Conway    SC     USA    Circle   
3     7/31/22 21:11          Keyser    WV     USA     Light   
4     7/31/22 20:43        gressham    OR     USA   Unknown   
...             ...             ...   ...     ...       ...   
2126  12/1/22 18:00       Snowflake    AZ     USA  Triangle   
2127  12/1/22 15:33             NaN   NaN     USA     Cross   
2128  12/1/22 14:10         El paso    TX     USA      Oval   
2129  12/1/22 06:20     CHARLESTOWN    IN     USA     Light   
2130  12/1/22 03:59       Mansfield    TX     USA  Changing   

                Duration                                            Summary  \
0              4 minutes            Bright circle light moving through sky.   
1                    NaN  Strange red, blue and black craft in the sky a...   
2     Less than a minute  Round bright white light go in a straight line...   
3              2 minutes                Saw an orb that interacted with me.   
4     less than a minute  A single soft red light that pulsed three time...   
...                  ...                                                ...   
2126           2 minutes  I grew up on a 40 acre parcel of land with thr...   
2127                 NaN  I observed some type of phenomenon that I was ...   
2128         Ten minutes  It appeared out of nowhere & slowly moved acro...   
2129      2 to 3 minutes  Bright object was seen arriving and silently h...   
2130          60 seconds  Looked like a drone but moved way too fast the...   

        Posted Images  ReportNum  
0       9/9/22    Yes          1  
1       9/9/22    Yes          2  
2       9/9/22    NaN          4  
3       9/9/22    NaN          5  
4       9/9/22    NaN          6  
...        ...    ...        ...  
2126  12/22/22    NaN        153  
2127  12/22/22    Yes        154  
2128  12/22/22    NaN        155  
2129  12/22/22    Yes        156  
2130  12/22/22    NaN        157  

[2131 rows x 10 columns]

In [25]:
combined_df = pd.merge(summary,unique_df)
combined_df

Date            City State Country     Shape  \
0     7/31/22 21:59  West Lafayette    OH     USA     Light   
1     7/31/22 21:45      Burr Ridge    IL     USA     Other   
2     7/31/22 21:15          Conway    SC     USA    Circle   
3     7/31/22 21:11          Keyser    WV     USA     Light   
4     7/31/22 20:43        gressham    OR     USA   Unknown   
...             ...             ...   ...     ...       ...   
2073  12/1/22 18:10       Blakeslee    OH     USA    Sphere   
2074  12/1/22 18:00       Snowflake    AZ     USA  Triangle   
2075  12/1/22 14:10         El paso    TX     USA      Oval   
2076  12/1/22 06:20     CHARLESTOWN    IN     USA     Light   
2077  12/1/22 03:59       Mansfield    TX     USA  Changing   

                Duration                                            Summary  \
0              4 minutes            Bright circle light moving through sky.   
1                    NaN  Strange red, blue and black craft in the sky a...   
2     Less than a minute  Round bright white light go in a straight line...   
3              2 minutes                Saw an orb that interacted with me.   
4     less than a minute  A single soft red light that pulsed three time...   
...                  ...                                                ...   
2073           5 minutes  Bright spherical object moving linear west to ...   
2074           2 minutes  I grew up on a 40 acre parcel of land with thr...   
2075         Ten minutes  It appeared out of nowhere & slowly moved acro...   
2076      2 to 3 minutes  Bright object was seen arriving and silently h...   
2077          60 seconds  Looked like a drone but moved way too fast the...   

        Posted Images  ReportNum            Location Latitude Longitude  
0       9/9/22    Yes          1  West Lafayette, OH  40.2753   -81.751  
1       9/9/22    Yes          2      Burr Ridge, IL  41.7489  -87.9184  
2       9/9/22    NaN          4          Conway, SC   33.836  -79.0478  
3       9/9/22    NaN          5          Keyser, WV  39.4409  -78.9739  
4       9/9/22    NaN          6        gressham, OR  45.5045  -122.436  
...        ...    ...        ...                 ...      ...       ...  
2073  12/22/22    Yes        152       Blakeslee, OH  41.5242  -84.7302  
2074  12/22/22    NaN        153       Snowflake, AZ  34.5134  -110.078  
2075  12/22/22    NaN        155         El paso, TX  31.7619  -106.485  
2076  12/22/22    Yes        156     CHARLESTOWN, IN  38.4531  -85.6702  
2077  12/22/22    NaN        157       Mansfield, TX  32.5632  -97.1417  

[2078 rows x 13 columns]